# Prepare SARS-CoV-2 dataset for indexing

In [1]:
import pandas as pd
from pathlib import Path
import os
import gzip
import json

# input_data = Path('data/sars-cov-2-genbank.json.gz')
input_data = Path('data/o.json.gz')
minimum_length = 29700
maximum_length = 31000
max_percent_ns = 10

genome_data = []
with gzip.open(input_data, 'rt') as fh:
    for line in fh:
        genome_entry = json.loads(line)
        genome_length = int(genome_entry['length'])
        
        if genome_length > minimum_length and genome_length < maximum_length:
            genome_ns_count = genome_entry['sequence'].upper().count('N')
            genome_ns_percent = 100 * (genome_ns_count / genome_length)
            
            if genome_ns_percent < max_percent_ns:
                genome_data.append({
                    'genbank_accession': genome_entry['genbank_accession'],
                    'strain': genome_entry['genbank_accession'],
                    'region': genome_entry['region'],
                    'location': genome_entry['location'],
                    'collection_date': genome_entry['collected'],
                    'submitted_date': genome_entry['submitted'],
                    'host': genome_entry['host'],
                    'isolation_source': genome_entry['isolation_source'],
                    'biosample_accession': genome_entry['biosample_accession'],
                    'length': genome_entry['length'],
                    'count_ns': genome_ns_count,
                    'percent_ns': genome_ns_percent,
                    'sequence': genome_entry['sequence']
                })
                
metadata_df = pd.DataFrame(genome_data)
metadata_df

,genbank_accession,strain,region,location,collection_date,submitted_date,host,isolation_source,biosample_accession,length,count_ns,percent_ns,sequence
0,NC_045512,NC_045512,Asia,China,2019-12,2020-01-13T00:00:00Z,Homo sapiens,,,29903,0,0.000000,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
1,MN908947,MN908947,Asia,China,2019-12,2020-01-12T00:00:00Z,Homo sapiens,,,29903,0,0.000000,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
2,MW898809,MW898809,Asia,Iran,2019-12-12,2021-04-12T00:00:00Z,Homo sapiens,,,29808,0,0.000000,AAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAA...
3,MT019529,MT019529,Asia,"China: Hubei, Wuhan",2019-12-23,2020-02-05T00:00:00Z,Homo sapiens,lung,,29899,0,0.000000,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
4,LR757995,LR757995,Asia,China:Wuhan,2019-12-26,2020-02-01T00:00:00Z,Homo sapiens,,SAMEA6507893,29872,0,0.000000,TTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,LR860756,LR860756,Europe,Switzerland:Graub###nden,2020,2020-07-20T00:00:00Z,Homo sapiens,,SAMEA7015170,29903,237,0.792563,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNACCAACCAACTTTCGA...
391,LR860757,LR860757,Europe,Switzerland:Uri,2020,2020-07-20T00:00:00Z,Homo sapiens,,SAMEA7015172,29903,488,1.631943,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNACCAACCAACTTTCGA...
392,LR860758,LR860758,Europe,Switzerland:Zurich,2020,2020-07-20T00:00:00Z,Homo sapiens,,SAMEA7015167,29903,398,1.330970,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNACCAACCAACTTTCGA...
393,LR860759,LR860759,Europe,Switzerland:Basel-Landschaft,2020,2020-07-20T00:00:00Z,Homo sapiens,,SAMEA7015171,29903,554,1.852657,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNACCAACCAACTTTCGA...


# Check for duplicate accessions

In [2]:
grouped = metadata_df.groupby('genbank_accession').agg({'genbank_accession': 'count'}).rename(
    {'genbank_accession': 'count'}, axis='columns')
duplicates = grouped[grouped['count'] > 1]
if len(duplicates) > 0:
    raise Exception(f'There are {len(duplicates)} duplicate accessions in data')
duplicates

,count
genbank_accession,


Looks like no duplicate accessions, so we are free to set the accession as the index.

In [3]:
metadata_df = metadata_df.set_index('genbank_accession', drop=False)
metadata_df.head(2)

,genbank_accession,strain,region,location,collection_date,submitted_date,host,isolation_source,biosample_accession,length,count_ns,percent_ns,sequence
genbank_accession,,,,,,,,,,,,,
NC_045512,NC_045512,NC_045512,Asia,China,2019-12,2020-01-13T00:00:00Z,Homo sapiens,,,29903,0,0.0,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
MN908947,MN908947,MN908947,Asia,China,2019-12,2020-01-12T00:00:00Z,Homo sapiens,,,29903,0,0.0,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...


# Write sequences to file

In [4]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

output_sequences_dir = Path('data/fasta')
output_sequences_all = Path('data/all.fasta')

if not output_sequences_dir.exists():
    os.mkdir(output_sequences_dir)
    
sequences_series = metadata_df.apply(lambda x: SeqRecord(Seq(x['sequence']), id=x['genbank_accession'],
                                                         description=''),
                                     axis='columns')

with open(output_sequences_all, 'w') as oha:
    for record in sequences_series:
        filename = f'{record.id}.fasta'
        output_file = output_sequences_dir / filename

        with open(output_file, 'w') as oh:
            SeqIO.write(record, oh, 'fasta')
            
        # Write to all file for pangolin lineages
        SeqIO.write(record, oha, 'fasta')
print(f'Wrote {len(sequences_series)} files in {output_sequences_dir}')
print(f'Wrote {len(sequences_series)} sequences in concatenated fasta file: {output_sequences_all}')

Wrote 395 files in data/fasta
Wrote 395 sequences in concatenated fasta file: data/all.fasta


# Write metadata to file

In [7]:
metadata_to_file = metadata_df.drop(['sequence'], axis='columns')
metadata_to_file.to_csv('data/metadata-genbank.tsv', sep='\t')